In [34]:
import os
import sys
import comtypes.client
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [35]:
# True: Trabajará con el programa ETBAS que esté abierto
# False: Se abrirá el programa de manera Automaticade 
AttachToInstance = False


# True: para especificar manualmente la ruta a ETABS.exe
# False: se usará la última versión instalada de ETABS
SpecifyPath = True

# si el indicador anterior está en True, especifique la ruta a ETABS a continuación
ProgramPath = "C:\Program Files\Computers and Structures\ETABS 20\ETABS.exe"

# ruta completa del modelo 
# ajústelo a la ruta deseada de su modelo
APIPath = 'C:\CSi_ETABS_API_Example'
if not os.path.exists(APIPath):
    try:
        os.makedirs(APIPath)
    except OSError:
        pass
ModelPath = APIPath + os.sep + 'API_1-001.edb'

# crear objeto API helper
helper = comtypes.client.CreateObject('ETABSv1.Helper')
helper = helper.QueryInterface(comtypes.gen.ETABSv1.cHelper)

if AttachToInstance:
    try:
        # obtener el objeto ETABS activo
        myETABSObject = helper.GetObject("CSI.ETABS.API.ETABSObject") 
    except (OSError, comtypes.COMError):
        print("No running instance of the program found or failed to attach.")
        sys.exit(-1)
else:
    if SpecifyPath:
        try:
            # crear una instancia del objeto ETABS a partir de la ruta especificada
            myETABSObject = helper.CreateObject(ProgramPath)
        except (OSError, comtypes.COMError):
            print("Cannot start a new instance of the program from " + ProgramPath)
            sys.exit(-1)
    else:
        try: 
            # crear una instancia del objeto ETABS a partir del último ETABS instalado
            myETABSObject = helper.CreateObjectProgID("CSI.ETABS.API.ETABSObject") 
        except (OSError, comtypes.COMError):
            print("Cannot start a new instance of the program.")
            sys.exit(-1)

    # Comenzar la aplicacion de ETABS
    myETABSObject.ApplicationStart()

# Crear un objeto SapModel
ret = SapModel = myETABSObject.SapModel

# Iniciar Modelo
ret = SapModel.InitializeNewModel()

In [36]:
#crear nuevo modelo en blanco
ret = SapModel.File.NewBlank()

In [37]:
#cambiar a unidades ton_m
Tonf_m_C = 12
ret = SapModel.SetPresentUnits(Tonf_m_C)

In [38]:
# Definicion de material Concreto
fc = 280
material_type = 2
material_name = "fc=" + str(fc) + "kg/cm2"    # fc=280kg/cm2
ret = SapModel.PropMaterial.SetMaterial(material_name, material_type)
# definir las propiedades mecanicas del material
E = 150000*fc**0.5          
v = 0.2
CT = 0.0000055
ret = SapModel.PropMaterial.SetMPIsotropic(material_name, E, v, CT)

In [39]:
# crear seccion de viga y columna
# Viga
b_v = 0.3
h_v = 0.6
nombre_viga = "V " + str(b_v) + "x" + str(h_v)          # V 0.3x0.6
ret = SapModel.PropFrame.SetRectangle(nombre_viga, material_name, h_v , b_v)


# Columna
b_c = 0.45
h_c = 0.45
nombre_columna = "C " + str(b_c) + "x" + str(h_c)       # C 0.45x0.45
ret = SapModel.PropFrame.SetRectangle(nombre_columna, material_name, h_c , b_c)

In [40]:
h = 5       # m  Altura de las columnas
L1 = 2      # m  Longitud del primer paño
paños = 18   #    Numero de paños

In [41]:
# columnas
for i in range(paños+1):
    FrameName = ' '
    [FrameName, ret] = SapModel.FrameObj.AddByCoord(i*L1,0,0,i*L1,0,h, FrameName, nombre_columna, '', 'Global')

# vigas
for i in range(paños):
    FrameName = ' '
    [FrameName, ret] = SapModel.FrameObj.AddByCoord(i*L1,0,h,(i+1)*L1,0,h, FrameName, nombre_viga, '', 'Global')

# Muro
name_wall = " "
x = [0, L1, L1, 0]
y = [0, 0, 0, 0]
z = [0, 0, h, h]
ret = SapModel.AreaObj.AddByCoord(4, x, y, z, name_wall)

name_wall = " "
x1 = [(paños-1)*L1, paños*L1, paños*L1, (paños-1)*L1]
y1 = [0, 0, 0, 0]
z1 = [0, 0, h, h]
ret = SapModel.AreaObj.AddByCoord(4, x1, y1, z1, name_wall)

In [42]:
#actualizar vista, actualizar (inicializar) zoom
ret = SapModel.View.RefreshView(0, False)

In [43]:
# asignar resitrcciones
Restraint = [True, True, True, True, True, True]        # empotramiento
NR = paños +1
for i in range(NR):
    PointNamei, PointNamej = ' ', ' '
    [PointNamei ,PointNamej, ret] = SapModel.FrameObj.GetPoints(str(i+1), PointNamei, PointNamej)
    ret = SapModel.PointObj.SetRestraint(PointNamei, Restraint)

#actualizar vista, actualizar (inicializar) zoom
ret = SapModel.View.RefreshView(0, False)

In [44]:
# asignar cargas
# crear un Load Pattern
ret = SapModel.LoadPatterns.Add("FH", 8, 0, True)

# Asignar carga distribuida Dead y Live
## Asignando cargas a la primera viga
for i in range(paños):
    ret = SapModel.FrameObj.SetLoadDistributed(str(paños+2+i), "Dead", 1, 2, 0, 1, -0.4, -0.4, "Local")
    ret = SapModel.FrameObj.SetLoadDistributed(str(paños+2+i), "Live", 1, 2, 0, 1, -0.25, -0.25, "Local")

In [46]:
# Asignar carga puntual
## extremo izquierdo - primera columna
PointName1 = ' '
PointName2 = ' '
[PointName1, PointName2, ret] = SapModel.FrameObj.GetPoints("1", PointName1, PointName2)
PointLoadValue = [1,0,0,0,0,0]
ret = SapModel.PointObj.SetLoadForce(PointName2, 'FH', PointLoadValue)